# PPFQ

### Data Mining

-48-

Buscamos patrones o indicadores que nos brinden señales.

Ejemplo: dos indicadores

objetivo: investigar entre muchos indicadores, correlaciones contra el precio, o el retorno futuro del mismo.

"Testear bot de trading del tipo trend-following, con cruce de medias y oscilador tipo RSI."

In [39]:
from dotenv import load_dotenv
import os
from apps.forex_api import  alpha_vantage_fx_api
import pandas as pd
import numpy as np


load_dotenv()
TOKEN = os.environ["TOKEN_AV"]
data = alpha_vantage_fx_api("FX_WEEKLY", "EUR", "USD", TOKEN)

In [40]:
# PARAMETROS INCIALES
ruedas = 14
par = "EUR/USD"
fast, slow = 20, 60
sigma = 40

In [41]:
# AJUSTE DE DATA
new_labels = ['Open', "High", "Low", "Close"]

data.rename(dict(zip(data.columns, new_labels)), axis=1, inplace=True)

data = data.iloc[-1::-1]


In [42]:
# cruce de SMAs
def cruce_sma_fast_slow(data, fast, slow):
    dataux = pd.DataFrame()
    dataux[f"SMA_{fast}"] = data.Close.rolling(fast).mean()
    dataux[f"SMA_{slow}"] = data.Close.rolling(slow).mean()
    data.loc[:, f"Cruce_{fast}_{slow}"] = dataux[f"SMA_{fast}"] / dataux[f"SMA_{slow}"] *100 -100
    return data

data = cruce_sma_fast_slow(data, fast, slow)


# calculo rsi
def rsi(data, ruedas):
    dataux = pd.DataFrame()
    dataux["dif"] = data.Close.diff()
    dataux["win"] = np.where(dataux["dif"] > 0, dataux["dif"], 0)
    dataux["loss"] = np.where(dataux["dif"] < 0, abs(dataux["dif"]), 0)
    dataux["ema_win"] = dataux.win.ewm(alpha=1/ruedas).mean()
    dataux["ema_loss"] = dataux.loss.ewm(alpha=1/ruedas).mean()
    dataux["rs"] = dataux.ema_win / dataux.ema_loss
    data.loc[:, "rsi"] = 100 - (100 / (1+ dataux.rs))
    return data

data = rsi(data, ruedas)


# Calculo volatilidad
def volatilidad(data, sigma):
    data.loc[:, "variacion"] = data.Close.pct_change() *100
    data.loc[:, f"sigma_{sigma}"] = data["variacion"].rolling(sigma).std() * (sigma **0.5)
    return data

data = volatilidad(data, sigma)

def ajustar_datos(data):
    return data.dropna().round(4)

data = ajustar_datos(data)

data


,Open,High,Low,Close,Cruce_20_60,rsi,variacion,sigma_40
2004-10-22,1.2472,1.2689,1.2446,1.2684,1.0755,63.0073,1.6835,7.8421
2004-10-29,1.2675,1.2841,1.2626,1.2783,1.1627,65.1131,0.7805,7.7423
2004-11-05,1.2740,1.2950,1.2652,1.2942,1.2641,68.2400,1.2438,7.7802
2004-11-12,1.2913,1.3005,1.2847,1.2979,1.3839,68.9378,0.2859,7.5752
2004-11-19,1.2970,1.3075,1.2913,1.3015,1.4540,69.6367,0.2774,7.5750
...,...,...,...,...,...,...,...,...
2022-09-30,0.9674,0.9854,0.9532,0.9800,-6.5164,31.0043,1.1383,8.2084
2022-10-07,0.9795,1.0000,0.9725,0.9741,-6.5986,29.7558,-0.6051,8.1734
2022-10-14,0.9738,0.9809,0.9629,0.9719,-6.7813,29.2847,-0.2258,8.1684
2022-10-21,0.9722,0.9876,0.9703,0.9861,-6.9002,36.2959,1.4611,8.3359
